### 아마존 리뷰 정보 가져오기
1. 사용자 정보
2. 평점
3. 날짜
4. 지역 정보, 몇 명에게 도움이 되었다.

In [21]:
import os, warnings
import re
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

### 물품 검색
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('./chromedriver_90')
url = 'https://www.amazon.com/'
driver.get(url)

sel_search = driver.find_element_by_xpath('//*[@id="twotabsearchtextbox"]')
sel_button = driver.find_element_by_xpath('//*[@id="nav-search-submit-button"]')

time.sleep(1)
sel_search.clear()   # 검색창 clear
time.sleep(1)
sel_search.send_keys('earphone')   # 이어폰 검색
time.sleep(1)
sel_button.click()   # 검색버튼 클릭
time.sleep(1)

In [22]:
### 상품 목록 중 첫 번째 상품 선택
sel_a = driver.find_elements_by_xpath('//*[@id="search"]/div[1]/div/div[1]/div/span[3]/div[2]')

for i in sel_a:
    sel_b = i.find_elements_by_tag_name('h2')
    for j in sel_b:
        if j.text:
            try:
                print('제품명 : ', j.text)
                print('url : ', j.find_element_by_tag_name('a').get_attribute('href'))
                print()
                one_url = j.find_element_by_tag_name('a').get_attribute('href')
                break
            except:
                print('url : Fail')
                print()
                continue
    break
    
time.sleep(1)
driver.get(one_url)
time.sleep(1)

## 리뷰 창 띄우기
rating = driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]')
print(rating.text)
rating.click()
time.sleep(2)

## 모든 리뷰 보기 창
all_review = driver.find_element_by_xpath('//*[@id="reviews-medley-footer"]/div[2]/a')
all_review.click()
time.sleep(1)

제품명 :  Amazon Basics In-Ear Wired Headphones Earbuds with Microphone, Red
url :  https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A049053719V83R4ZNWDQA&url=%2FAmazonBasics-17E13RD-Ear-Headphones-Mic%2Fdp%2FB07HH223KV%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dearphones%26qid%3D1624258995%26sr%3D8-1-spons%26psc%3D1&qualifier=1624258995&id=7076510246670124&widgetName=sp_atf

6,541 ratings


In [24]:
reviews = []
names = []
stars = []
dates = []
countrys = []
helpfuls = []
            
### 정보 긁어오기
for pg in range(5):    # 페이지 수
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    
    next_button = driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a')
    print('==================')
    print('page', pg+1, 'Crawling...')
    print('==================')
    
    review_list = soup.findAll('span', class_='a-size-base review-text review-text-content')
    for review in review_list:
        reviews.append(review.text.strip())

    name_list = soup.findAll('span', class_='a-profile-name')
    for name in name_list[2:]:
        names.append(name.text)

    star_list = soup.findAll('a', class_='a-link-normal')
    for star in star_list:
        star_1 = star.findAll('span', class_='a-icon-alt')
        for st in star_1:
            stars.append(st.text)

    date_list = soup.findAll('span', class_="a-size-base a-color-secondary review-date")
    for date in date_list[2:]:
        countrys.append(date.text.split(' on ')[0])
        dates.append(date.text.split(' on ')[1])

    helpful_list = soup.findAll('span', class_="cr-vote")
    for helpful in helpful_list:
        help_1 = helpful.text.split('\n')
        for one in range(1, len(help_1), 6):
            if not help_1[one]:  
                helpfuls.append('0 people found this helpful')
            else:
                helpfuls.append(help_1[one])
    
    time.sleep(3)
    next_button.click()
    time.sleep(3)  
#     driver.execute_script('window.scrollTo(0, 3500);')

print('======================')
print('Done')
print('======================')

page 1 Crawling...
page 2 Crawling...
page 3 Crawling...
page 4 Crawling...
page 5 Crawling...
Done


In [25]:
## 파일로 변환
total_review = {'날짜':dates , '국가':countrys , '이름':names , '평점':stars ,
                '리뷰':reviews , '도움':helpfuls}

total_review = pd.DataFrame(total_review)
total_review

,날짜,국가,이름,평점,리뷰,도움
0,"March 22, 2019",Reviewed in the United States,Rick Layton,2.0 out of 5 stars,I made a mistake. The original Earpods that ca...,87 people found this helpful
1,"April 3, 2019",Reviewed in the United States,Emily,5.0 out of 5 stars,These are very inexpensive headphones but they...,63 people found this helpful
2,"April 2, 2019",Reviewed in the United States,romeo,1.0 out of 5 stars,"The cables are very flimsy and thin, similar t...",65 people found this helpful
3,"February 11, 2019",Reviewed in the United States,Ed W.,5.0 out of 5 stars,The Amazon basic ear headphones works in the i...,40 people found this helpful
4,"February 28, 2019",Reviewed in the United States,Sunderrajan,4.0 out of 5 stars,I liked the color - my sons can't tell me that...,39 people found this helpful
5,"March 22, 2019",Reviewed in the United States,Brandy C,5.0 out of 5 stars,My 13 year old uses these on a daily basis. Sh...,30 people found this helpful
6,"February 11, 2019",Reviewed in the United States,Daniel,2.0 out of 5 stars,They produce sound but it comes with a lot of ...,26 people found this helpful
7,"March 26, 2019",Reviewed in the United States,Vee,4.0 out of 5 stars,These headphones are surprisingly nice for the...,21 people found this helpful
8,"July 21, 2020",Reviewed in the United States,Bob K,2.0 out of 5 stars,"I usually buy Skullcandy Ink'd, but bought the...",7 people found this helpful
9,"April 12, 2020",Reviewed in the United States,Robert Beauregard,5.0 out of 5 stars,Who'd have thunk it? Noise-canceling earbuds f...,4 people found this helpful


In [26]:
total_review.to_csv("Amazon_review.csv", index=False)

## 확인
print(os.getcwd())  # 현재 위치
print(os.listdir(os.getcwd()))

C:\Users\user\LikeLion_AI
['.ipynb_checkpoints', 'Amazon_review.csv', 'Amazon_review.ipynb', 'Amazon_selenium.ipynb', 'chromedriver_90.exe', 'data', 'dog_e.png', 'LLAI_06_09.ipynb', 'LLAI_06_14.ipynb', 'LLAI_06_16.ipynb', 'LLAI_06_17.ipynb', 'LLAI_06_18.ipynb', 'LLAI_06_21.ipynb', 'url.csv', 'Web_test.ipynb', '네이버 영화 정보.csv', '달라스바이어스클럽_리뷰_06_17.csv', '스파이더맨_리뷰_06_17.csv', '의료용어.csv', '인기종목 정보.csv', '회사명과 웹사이트.csv']


### 길이를 확인해서 간결하게 확인해보기

In [ ]:
print(len(dates))
print('--------')
print(len(countrys))
print('--------')
print(len(names))
print('--------')
print(len(stars))
print('--------')
print(len(reviews))
print('--------')
print(len(helpfuls))
print('--------')

# print(dates)
# print('--------')
# print(countrys)
# print('--------')
# print(names)
# print('--------')
# print(stars)
# print('--------')
# print(review_list)
# print('--------')
# print(helpfuls)
# print('--------')